In [1]:
data_date = "2018_06_27_13_49_07"
data_dir = "/home/pleroy/DATA/2018_06_27_LETG/2018_06_27/jde/" + data_date
# selection in record
firstFile = 0
nbFiles = 379

# Initialization

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

import sys
import gc

sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.readdata import *

%load_ext autoreload
%autoreload 2

sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

# Parameters

In [3]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )
params.print()

requestedTRamp 1200.0
configuredTRamp 1200.0
startFrequency 5725000000.0
stopFrequency 5875000000.0
frequencyBand 150000000.0
frequencyDevReq 50000.0
frequencyDevConf 50001.1
numberOfSteps 3000
waveformType 1
rampsPerBuffer 375
bufferSize 9000000.0
buffersPerFile 2
rampsPerFile 750
fileSize 18000000.0
samplingFrequency 10000000.0
samplesPerRamp 12000
skipNSamples 0


In [4]:
buffersPerFile = params.buffersPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

print( "buffersPerFile = {}, samplesPerRamp = {}, rampsPerFile = {}".format(
    buffersPerFile, samplesPerRamp, rampsPerFile )
     )

lastFile = firstFile + nbFiles - 1
firstBuffer = firstFile * buffersPerFile

buffersPerFile = 2, samplesPerRamp = 12000, rampsPerFile = 750


In [5]:
lastFile

378

# Read the data

In [6]:
scalingFactor = 2 / 65535
offset = -32768

In [7]:
timeSerie_A = np.zeros( samplesPerFile )
adc_A = np.zeros( (nbFiles, samplesPerFile) )

In [8]:
lastBuffer = (nbFiles-1) * buffersPerFile + firstBuffer
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * buffersPerFile + firstBuffer) )
    filename = data_dir + "/record" + nb + ".bin"
    readFileADLINKCh0( filename, samplesPerFile, timeSerie_A )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBuffer))
    
    adc_A[ k, : ] = (timeSerie_A + offset) * scalingFactor

k = 0, block = 0 / 756
k = 1, block = 2 / 756
k = 2, block = 4 / 756
k = 3, block = 6 / 756
k = 4, block = 8 / 756
k = 5, block = 10 / 756
k = 6, block = 12 / 756
k = 7, block = 14 / 756
k = 8, block = 16 / 756
k = 9, block = 18 / 756
k = 10, block = 20 / 756
k = 11, block = 22 / 756
k = 12, block = 24 / 756
k = 13, block = 26 / 756
k = 14, block = 28 / 756
k = 15, block = 30 / 756
k = 16, block = 32 / 756
k = 17, block = 34 / 756
k = 18, block = 36 / 756
k = 19, block = 38 / 756
k = 20, block = 40 / 756
k = 21, block = 42 / 756
k = 22, block = 44 / 756
k = 23, block = 46 / 756
k = 24, block = 48 / 756
k = 25, block = 50 / 756
k = 26, block = 52 / 756
k = 27, block = 54 / 756
k = 28, block = 56 / 756
k = 29, block = 58 / 756
k = 30, block = 60 / 756
k = 31, block = 62 / 756
k = 32, block = 64 / 756
k = 33, block = 66 / 756
k = 34, block = 68 / 756
k = 35, block = 70 / 756
k = 36, block = 72 / 756
k = 37, block = 74 / 756
k = 38, block = 76 / 756
k = 39, block = 78 / 756
k = 40, block =

k = 310, block = 620 / 756
k = 311, block = 622 / 756
k = 312, block = 624 / 756
k = 313, block = 626 / 756
k = 314, block = 628 / 756
k = 315, block = 630 / 756
k = 316, block = 632 / 756
k = 317, block = 634 / 756
k = 318, block = 636 / 756
k = 319, block = 638 / 756
k = 320, block = 640 / 756
k = 321, block = 642 / 756
k = 322, block = 644 / 756
k = 323, block = 646 / 756
k = 324, block = 648 / 756
k = 325, block = 650 / 756
k = 326, block = 652 / 756
k = 327, block = 654 / 756
k = 328, block = 656 / 756
k = 329, block = 658 / 756
k = 330, block = 660 / 756
k = 331, block = 662 / 756
k = 332, block = 664 / 756
k = 333, block = 666 / 756
k = 334, block = 668 / 756
k = 335, block = 670 / 756
k = 336, block = 672 / 756
k = 337, block = 674 / 756
k = 338, block = 676 / 756
k = 339, block = 678 / 756
k = 340, block = 680 / 756
k = 341, block = 682 / 756
k = 342, block = 684 / 756
k = 343, block = 686 / 756
k = 344, block = 688 / 756
k = 345, block = 690 / 756
k = 346, block = 692 / 756
k

In [9]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)

**Check that the first ramp is an up ramp**

In [10]:
plt.figure()
plt.plot( A_reshaped[0, :] )
title = "first acquisition " + data_date
plt.title(title)
plt.grid()
plt.savefig( data_dir + "/" + title + ".png", bbox_inches='tight')

**Set the shifted flag properly**

In [11]:
rampDownFirst = 1

# Build RD

In [12]:
#RDc = build_rd_from_data( params, A_reshaped, shifted )
withHanning = 1
RDc = build_rd_from_data_rampDown( params, A_reshaped, rampDownFirst, withHanning )

shape of the samples matrix = (284250, 12000)
with Hanning window
ramp down first in the data files


In [13]:
A_reshaped = None
adc_A = None
gc.collect()

2430

In [14]:
coupling_RD = np.average(RDc, 0)
if withHanning:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown_hanning".format(firstFile, lastFile), coupling_RD )
else:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown".format(firstFile, lastFile), coupling_RD )

In [15]:
plt.figure()
plt.plot(np.abs(coupling_RD), 'k', label='coupling')
plt.legend()
plt.grid()

In [16]:
# save RD data
RD = np.fft.ifftshift(RDc, 1)

In [17]:
np.save(data_dir + '/np_fft_ifftshift(RD1, 1)_{}_{}'.format(firstFile, lastFile), RD) 